In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
load_dotenv(override=True)

True

## Agent State

In [2]:
from typing import TypedDict, Annotated, Union, List
import operator
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage, HumanMessage

In [3]:
class customState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

## Function Calling

In [4]:
tavily_search = TavilySearch(max_results =1)

In [5]:
@tool
def triple_num(num: float):
    """
    This tool accepts a number, triples it and returns the tripled number
    """
    return num * 3

In [6]:
tools = [tavily_search, triple_num]

In [7]:
llm = ChatOpenAI(model="gpt-4.1", temperature = 0.).bind_tools(tools)

In [8]:
def reasoning_node(state: customState):
    output = llm.invoke(state["messages"])
    state["messages"].append(output)
    return state

#### Testing 

In [9]:
state_dict = {"messages": [HumanMessage(content = "What is temperature in Tokyo, triple it and return the updated result")]}
state_dict = reasoning_node(state_dict)

In [10]:
state_dict#.tool_calls

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

## Tool Node

In [11]:
from langgraph.prebuilt import ToolNode

In [12]:
tool_node = ToolNode(tools = tools)

In [13]:
def tool_node_handler(state: customState):
    output = tool_node.invoke(state["messages"])
    state["messages"].append(*output)
    return state

#### Testing

In [14]:
state_dict = tool_node_handler(state_dict)

In [15]:
state_dict

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

In [16]:
state_dict = reasoning_node(state_dict)

In [17]:
state_dict

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

In [18]:
state_dict = tool_node_handler(state_dict)

In [19]:
state_dict

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

In [20]:
state_dict = reasoning_node(state_dict)

In [21]:
state_dict

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

In [22]:
state_dict = tool_node_handler(state_dict)

In [23]:
state_dict

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

In [24]:
state_dict = reasoning_node(state_dict)

In [25]:
state_dict

{'messages': [HumanMessage(content='What is temperature in Tokyo, triple it and return the updated result', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9arhHOnoLgL7dHgAil5l3BuN', 'function': {'arguments': '{"query":"current temperature in Tokyo"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 809, 'total_tokens': 828, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-Bomn5k5MfpISt74ZaVY2ukpG8EUGu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cb86dcca-d2c6-4e40-b463-838f4787cda6-0', tool_calls=[{'name': 'tavily_search', 'args': {'query'

## Nodes

In [29]:
from langgraph.prebuilt import ToolNode

In [30]:
agent.invoke({"input" : "What is agent Memory"})

KeyError: "Input to PromptTemplate is missing variables {'tools', 'tool_names', 'agent_scratchpad', 'input'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['messages', 'is_last_step', 'remaining_steps']\nNote: if you intended {tools} to be part of the string and not a variable, please escape it with double curly braces like: '{{tools}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:
def agent_reasoning_node(state: AgentState):
    agent_outcome = agent.invoke({"input" : "What is agent Memory"})
    

In [14]:
print(react_prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


True